# 4.2 Lab: split, apply, and combine with chipotle

---

In this lab you'll practice the split-apply-combine pattern common in pandas with a dataset on chipotle orders.

The chipotle orders data is messy: the column with ingredients in the order is a list of lists, which will need to be dealt with. This specifically will also be practice with long and wide format data.



---

## 1. Load the data and packages




In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('darkgrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [7]:
chip_file = '/Users/ryandunlap/Desktop/DSI-SF-2/datasets/chipotle_orders/chipotle.tsv'

# the chipotle data is a .tsv, which means "tab separated value". 
# This is just like a csv but the cells are separated by tabs.
# There is an argument in read_csv called delimiter, where you can specify
# the string that separates the tabs:
chip = pd.read_csv(chip_file, delimiter='\t')

In [8]:
chip.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


---

## 2. Create a sub-id for each order-id

We have an identifier for each order already in `order_id`, but no unique identifier for each _sub-order_ within the overall order.

Use grouping to assign sub-ids for orders.

In [39]:
def create_sub_id(subset):
    subset['sub_order_id'] = range(subset.shape[0])
    return subset

chip = pd.read_csv(chip_file, delimiter='\t')
chip = chip.groupby('order_id').apply(create_sub_id)

---

## 3. Clean price column 

This is in my opinion most easily done with apply.

In fact, I basically only use apply with custom functions despite groupby having some built-in functionality, because it is maximally flexible.

For operating on single columns (like when cleaning), the `.map()` function is useful. It operates on every element of the series/column.

In [40]:
def clean_price(series):
    series = series.replace("$","")
    return float(series)

chip.item_price = chip.item_price.map(clean_price)
chip.item_price

0        2.39
1        3.39
2        3.39
3        2.39
4       16.98
5       10.98
6        1.69
7       11.75
8        9.25
9        9.25
10       4.45
11       8.75
12       8.75
13      11.25
14       4.45
15       2.39
16       8.49
17       8.49
18       2.18
19       8.75
20       4.45
21       8.99
22       3.39
23      10.98
24       3.39
25       2.39
26       8.49
27       8.99
28       1.09
29       8.49
        ...  
4592    11.75
4593    11.75
4594    11.75
4595     8.75
4596     4.45
4597     1.25
4598     1.50
4599     8.75
4600     4.45
4601     1.25
4602     9.25
4603     9.25
4604     8.75
4605     4.45
4606     1.25
4607    11.75
4608    11.25
4609     1.25
4610    11.75
4611    11.25
4612     9.25
4613     2.15
4614     1.50
4615     8.75
4616     4.45
4617    11.75
4618    11.75
4619    11.25
4620     8.75
4621     8.75
Name: item_price, dtype: float64

---

## 4. Make a new categorical column for broader item type

You should have these types in the new column in your DataFrame:

    chips
    drink
    burrito
    taco
    salad
    
(Put `bowl` items into `burrito` category)

In [92]:
def label_cat(series):
    if 'chip' in series.lower():
        cat = 'chips'
    elif 'taco' in series.lower():
        cat = 'taco'
    elif 'burrito' in series.lower() or 'bowl' in series.lower():
        cat = 'burrito'
    elif 'salad' in series.lower():
        cat = 'salad'
    elif ('drink' in series.lower() or 'canned' in series.lower() or 'bottled' in series.lower() or
         'izze' in series.lower() or 'nantucket' in series.lower()):
        cat = 'drink'
    else:
        cat = 'WTF'
        
    return cat

chip['category'] = chip['item_name'].map(label_cat)
chip.head()

,order_id,quantity,item_name,choice_description,item_price,sub_order_id,category
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0,chips
1,1,1,Izze,[Clementine],3.39,1,drink
2,1,1,Nantucket Nectar,[Apple],3.39,2,drink
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3,chips
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0,burrito


---

## 5. Calculate the total price by `order_id` and append as column `order_total_price`

I find this is easiest/clearest with a grouped apply on the price and then a merge by `order_id` with the total price.

Hints:

- Merging DataFrames with series doesn't work, you need to merge 2 DataFrames.
- A series object coming out of a groupby with an apply will have the groupby as potentially hierarchical indices. Using `reset_index()` will turn these back into columns and also convert to a DataFrame, which can be used to merge on.

In [94]:
def item_total(series,axis=1):
    series['order_total'] = sum(series['item_price']*series['quantity'])
    return series

chip = chip.groupby(['order_id']).apply(item_total)

---

## 6. Make an adjusted item_price column

Some items have multiple orders per row, as indicated by the quantity. Adjust the price accordingly in a new column.

In [96]:
#Handled above

,order_id,quantity,item_name,choice_description,item_price,sub_order_id,category,order_total
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0,chips,11.56
1,1,1,Izze,[Clementine],3.39,1,drink,11.56
2,1,1,Nantucket Nectar,[Apple],3.39,2,drink,11.56
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3,chips,11.56
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0,burrito,33.96
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98,0,burrito,12.67
6,3,1,Side of Chips,NaN,1.69,1,chips,12.67
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75,0,burrito,21.00
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25,1,taco,21.00
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",9.25,0,burrito,13.70


---

## 7. What is the min, max, count, mean and standard deviation price per the `item_name`?

Pivot table would be good for this. Remember you can pass multiple functions into the `aggfunc` argument.

You'll have to calculate the count separately due to the quantity column and merge it in after!

In [104]:
my_func = [('min',np.min),('max',np.max),('count','count'),('mean',np.mean),('std',np.std)]
chip.groupby(['item_name'])['item_price'].agg(my_func).reset_index()
chip[chip.item_name=='6 Pack Soft Drink']

,order_id,quantity,item_name,choice_description,item_price,sub_order_id,category,order_total
298,129,1,6 Pack Soft Drink,[Sprite],6.49,2,drink,29.99
341,148,1,6 Pack Soft Drink,[Diet Coke],6.49,3,drink,30.94
357,154,1,6 Pack Soft Drink,[Coke],6.49,2,drink,18.19
388,168,1,6 Pack Soft Drink,[Diet Coke],6.49,1,drink,18.24
417,182,1,6 Pack Soft Drink,[Diet Coke],6.49,1,drink,20.19
553,230,1,6 Pack Soft Drink,[Diet Coke],6.49,3,drink,38.19
639,264,1,6 Pack Soft Drink,[Diet Coke],6.49,1,drink,43.49
721,298,1,6 Pack Soft Drink,[Nestea],6.49,0,drink,15.24
743,306,1,6 Pack Soft Drink,[Coke],6.49,1,drink,18.24
879,363,1,6 Pack Soft Drink,[Coke],6.49,1,drink,33.54


---

## 8. Plot the mean price of items against the count (popularity)

You have this info in your summary table from the previous question.

---

## 8.2 Plot the standard deviation of price of items against the count (popularity)


---

## 9. Parse the `choice_description` column into two new columns: `order_customizations` and `order_customization_id`

Here is what your inputs and outputs would look like for a hypotheical section of the DataFrame (I'm just showing some of the columns to give you an idea of what the output format will be):

**Input:**

```python
                                  choice_description     item_name  order_id  \
0                                       [Clementine]          Izze         1   
1  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   

   sub_order_id  
0             1  
1             2
```

**Output:**

```python
   order_customization_id order_customizations  \
0                       0           Clementine   
1                       0            Red Salsa   
2                       1          Black Beans   
3                       2            Guacamole   
4                       3           Sour Cream   

                                  choice_description     item_name  order_id  \
0                                       [Clementine]          Izze         1   
1  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
2  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
3  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
4  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   

   sub_order_id  
0             1  
1             2  
2             2  
3             2  
4             2 
```

Hints:

- The way I did it was to write my own function and pass it into apply. In this case there will be one item per group, since we have to do this parsing for every row, but you may be able to get a more "elegant" solution with `.iterrows()` if you want to try that out.
- Within a function that you are passing into `.apply()`, you can create a _new DataFrame and return that._ This is one of the things that makes apply so powerful, since you can essentially perform any operations you want on a subset of your original DataFrame as long as you return DataFrames/groups that can be recombined.

**Note: the function may take awhile to complete. apply isn't terribly efficient, especially with complicated operations per row like this.**

---

## 10. Calculate the std of adjusted price for broad category

You can handle these with a single function if you want, or another way if you prefer.

Just FYI, apply functions can have keyword arguments that you pass in when you call the apply chained to the groupby.

for example:

```python
def my_applier(df, my_kwarg='placeholder'):
    df['newcol_'+placeholder] = 1.
    return df
    
data = data.groupby('variable').apply(my_applier, my_kwarg='colsuffix').reset_index(drop=True)
```


---

## 11. Make a seaborn barplot of your price std by the broad type category

You'll need to lengthen this out with melt to have a column containing the information on the tier and price variables.